In [27]:
# Import tmdb api wrapper library
import tmdbsimple as tmdb

# Import JSON
import json

# import time for sleeping and scraping
import time

# Import the usual suspects
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns', 500)

# pickle for saving and loading scraped info
import pickle


from timeit import default_timer as timer


# Pandas profiling - great library for analyzing dataframes
import pandas_profiling as pp



from sklearn.preprocessing import MultiLabelBinarizer



In [4]:
with open('../data/processed/all_episodes.pkl', 'rb') as f:
    import_df = pickle.load(f)

In [5]:
import_df.shape

(1089243, 29)

In [6]:
import_df.head()

,created_by,episode_run_time,first_air_date,genres,id_show,in_production,last_air_date,name_show,networks,number_of_episodes,number_of_seasons,origin_country,original_language,original_name,popularity,production_companies,status,type,vote_average_show,vote_count_show,air_date,crew,episode_number,guest_stars,id_ep,name_ep,season_number,vote_average_ep,vote_count_ep
0,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-12,"[{'id': 10707, 'credit_id': '525389cd19c295794...",1.0,[],34326.0,From the Dawn of History to the Normal Conquest,1.0,0.0,0.0
1,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-19,"[{'id': 10707, 'credit_id': '525389cd19c295794...",2.0,[],34328.0,Richard the Lionheart to Robin the Hood,1.0,0.0,0.0
2,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-26,"[{'id': 10707, 'credit_id': '525389cd19c295794...",3.0,[],34327.0,Edward the First to Richard the Last,1.0,0.0,0.0
3,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-02-02,"[{'id': 10707, 'credit_id': '525389cd19c295794...",4.0,[],34331.0,Perkin Warbeck to Bloody Mary,1.0,0.0,0.0
4,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-02-09,"[{'id': 10707, 'credit_id': '525389cd19c295794...",5.0,[],34330.0,The Great and Glorious Age of Elizabeth,1.0,0.0,0.0


In [7]:
def extract_id(x):
    if type(x)==list:
        return [y['id'] for y in x]
    else:
        return []

In [8]:
import_df['crew_id'] = import_df.crew.apply(extract_id)
import_df['created_by_id'] = import_df.created_by.apply(extract_id)
import_df['networks_id'] = import_df.networks.apply(extract_id)
import_df['genres_id'] = import_df.genres.apply(extract_id)


In [9]:
import_df[~import_df.networks.apply(type).eq(list)].networks.value_counts()

Series([], Name: networks, dtype: int64)

In [10]:
import_df.dtypes

created_by               object
episode_run_time         object
first_air_date           object
genres                   object
id_show                   int64
in_production              bool
last_air_date            object
name_show                object
networks                 object
number_of_episodes      float64
number_of_seasons         int64
origin_country           object
original_language        object
original_name            object
popularity              float64
production_companies     object
status                   object
type                     object
vote_average_show       float64
vote_count_show           int64
air_date                 object
crew                     object
episode_number          float64
guest_stars              object
id_ep                   float64
name_ep                  object
season_number           float64
vote_average_ep         float64
vote_count_ep           float64
crew_id                  object
created_by_id            object
networks

In [11]:
import_df.isna().sum() / len(import_df)

created_by              0.000000
episode_run_time        0.000000
first_air_date          0.060515
genres                  0.000000
id_show                 0.000000
in_production           0.000000
last_air_date           0.053057
name_show               0.000000
networks                0.000000
number_of_episodes      0.000497
number_of_seasons       0.000000
origin_country          0.000000
original_language       0.000000
original_name           0.000000
popularity              0.000000
production_companies    0.000000
status                  0.000000
type                    0.000000
vote_average_show       0.000000
vote_count_show         0.000000
air_date                0.115473
crew                    0.026396
episode_number          0.026396
guest_stars             0.026396
id_ep                   0.026396
name_ep                 0.026396
season_number           0.026444
vote_average_ep         0.026396
vote_count_ep           0.026396
crew_id                 0.000000
created_by

In [12]:
len(import_df[import_df.vote_count_ep>0]) / len(import_df)

0.0469335125403606

In [13]:
import_df.head()

,created_by,episode_run_time,first_air_date,genres,id_show,in_production,last_air_date,name_show,networks,number_of_episodes,number_of_seasons,origin_country,original_language,original_name,popularity,production_companies,status,type,vote_average_show,vote_count_show,air_date,crew,episode_number,guest_stars,id_ep,name_ep,season_number,vote_average_ep,vote_count_ep,crew_id,created_by_id,networks_id,genres_id
0,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-12,"[{'id': 10707, 'credit_id': '525389cd19c295794...",1.0,[],34326.0,From the Dawn of History to the Normal Conquest,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
1,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-19,"[{'id': 10707, 'credit_id': '525389cd19c295794...",2.0,[],34328.0,Richard the Lionheart to Robin the Hood,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
2,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-26,"[{'id': 10707, 'credit_id': '525389cd19c295794...",3.0,[],34327.0,Edward the First to Richard the Last,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
3,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-02-02,"[{'id': 10707, 'credit_id': '525389cd19c295794...",4.0,[],34331.0,Perkin Warbeck to Bloody Mary,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
4,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-02-09,"[{'id': 10707, 'credit_id': '525389cd19c295794...",5.0,[],34330.0,The Great and Glorious Age of Elizabeth,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]


In [12]:
mlb = MultiLabelBinarizer()

s = import_df.genres_id

# s = s.loc[s.isnull()].apply(lambda x: [])

genres_id_df = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=import_df.index)
genres_id_df.shape

(1089243, 29)

In [13]:
s = import_df.created_by_id

created_by_id_df = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=import_df.index)
created_by_id_df.shape

(1089243, 7812)

In [14]:
s = import_df.crew_id

crew_id_df = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=import_df.index)
crew_id_df.shape

(1089243, 26142)

In [ ]:
s = import_df.networks_id

networks_id_df = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=import_df.index)
networks_id_df.shape

In [15]:
# Import api key
tmdb.API_KEY = open('../api_key', 'rt').read()

In [16]:
import_df.head()

,created_by,episode_run_time,first_air_date,genres,id_show,in_production,last_air_date,name_show,networks,number_of_episodes,number_of_seasons,origin_country,original_language,original_name,popularity,production_companies,status,type,vote_average_show,vote_count_show,air_date,crew,episode_number,guest_stars,id_ep,name_ep,season_number,vote_average_ep,vote_count_ep,crew_id,created_by_id,networks_id,genres_id
0,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-12,"[{'id': 10707, 'credit_id': '525389cd19c295794...",1.0,[],34326.0,From the Dawn of History to the Normal Conquest,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
1,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-19,"[{'id': 10707, 'credit_id': '525389cd19c295794...",2.0,[],34328.0,Richard the Lionheart to Robin the Hood,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
2,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-01-26,"[{'id': 10707, 'credit_id': '525389cd19c295794...",3.0,[],34327.0,Edward the First to Richard the Last,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
3,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-02-02,"[{'id': 10707, 'credit_id': '525389cd19c295794...",4.0,[],34331.0,Perkin Warbeck to Bloody Mary,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]
4,"[{'id': 383, 'credit_id': '525389d019c29579402...",[30],1969-01-12,[],601,True,1969-02-16,The Complete and Utter History of Britain,"[{'name': 'London Weekend Television', 'id': 1...",6.0,1,[GB],en,The Complete and Utter History of Britain,0.6,[],Ended,Scripted,0.0,0,1969-02-09,"[{'id': 10707, 'credit_id': '525389cd19c295794...",5.0,[],34330.0,The Great and Glorious Age of Elizabeth,1.0,0.0,0.0,"[10707, 383]","[383, 10707]",[127],[]


In [17]:
import_df.iloc[10000]

created_by                                                             []
episode_run_time                                                     [60]
first_air_date                                                 1994-07-24
genres                  [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...
id_show                                                               787
in_production                                                       False
last_air_date                                                  1998-07-05
name_show                                                        Wycliffe
networks                [{'name': 'ITV', 'id': 9, 'logo_path': '/5Rmw4...
number_of_episodes                                                     36
number_of_seasons                                                       5
origin_country                                                       [GB]
original_language                                                      en
original_name                         

In [18]:
a = tmdb.TV(42885)

In [19]:
a.external_ids()

{'id': 42885,
 'imdb_id': 'tt1145380',
 'freebase_mid': '/m/0j_m6xk',
 'freebase_id': None,
 'tvdb_id': None,
 'tvrage_id': None,
 'facebook_id': None,
 'instagram_id': None,
 'twitter_id': None}

In [20]:
tv_ep_ids = import_df.id_ep

In [ ]:
temp_list = []
count = 0
start = timer()

skipped_ids = []

for ep_id in tv_ep_ids.values:
    no_response = True
    attempt = 0
    while no_response:
        try:
            temp_list.append(tmdb.TV(int(ep_id)).external_ids()['imdb_id'])
            no_response = False
        except:
            time.sleep(1)
            attempt += 1
                # if trying same file multiple times, skip it
            if attempt>10:
                skipped_ids.append(int(ep_id))
                print('skipped: {}'.format(int(ep_id)))
                break
    count+=1
    
    if count%100==0: 
        print(len(imdb_id_list))
        print('seasons: {}'.format(season_count))
        end = timer()
        print('{:.2f} seconds'.format(end - start))
        start = timer()
        
    if count%10000==0:
        f_num = 0
        exists = True
        while(exists):
            f_num += 1
            exists = os.path.isfile(
                '../data/processed/tv_episodes/tv_episodes_imdb_{}.pkl'.format(f_num))
 
        if not exists:
            with open('../data/processed/tv_episodes/tv_episodes_imdb_{}.pkl'.format(f_num), 'wb') as f:
                pickle.dump(temp_list, f)
            print('saved to: ../data/processed/tv_episodes/tv_episodes_imdb_{}.pkl'.format(f_num))
            temp_list = []

skipped: 34339
skipped: 34334
skipped: 1053081
skipped: 1053082
skipped: 1053083
skipped: 1053084
skipped: 1053085
skipped: 1053086
skipped: 1053087


In [21]:
tmdb.TV(int(34331)).external_ids()['imdb_id']

'tt1758290'

In [26]:
tv_ep_ids[:10]

0    34326.0
1    34328.0
2    34327.0
3    34331.0
4    34330.0
5    34329.0
6    34333.0
7    34332.0
8    34335.0
9    34336.0
Name: id_ep, dtype: float64